In [4]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, translate
from IPython.core.display import display, HTML

In [5]:

spark= SparkSession.builder.appName('MLFlowRegression').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 16:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [6]:
# read the data to create spark data frame 
airBnBData = spark.read.csv('/Users/badr/Desktop/python/MLpySpark/ml-100k/listings.csv',header='true', inferSchema='true', multiLine='true',escape='"' )

In [8]:
# display all column that has integer value 
null_columns = [column for column in airBnBData.columns if airBnBData.where(col(column).isNull()).count() > 0]

In [7]:
# get the columns that is integer type 
from pyspark.sql.types import IntegerType , DoubleType
integer_type = [col.name for col in airBnBData.schema.fields if col.dataType == IntegerType()]
display(integer_type)

['id',
 'host_id',
 'host_listings_count',
 'host_total_listings_count',
 'accommodates',
 'bedrooms',
 'beds',
 'square_feet',
 'guests_included',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'calculated_host_listings_count']

In [8]:
# convert all integer values to double 
from pyspark.sql.functions import cast, translate
airBnBData_v1 = airBnBData

# airBnBData1 = airBnBData.select(*[col(col_name).cast('double').alias(col_name) for col_name in integer_type])

for col_name in integer_type:
   airBnBData_v1 = airBnBData_v1.withColumn(col_name, col(col_name).cast("double")) 

In [11]:
display(airBnBData_v1)

DataFrame[id: double, listing_url: string, scrape_id: bigint, last_scraped: date, name: string, summary: string, space: string, description: string, experiences_offered: string, neighborhood_overview: string, notes: string, transit: string, thumbnail_url: string, medium_url: string, picture_url: string, xl_picture_url: string, host_id: double, host_url: string, host_name: string, host_since: date, host_location: string, host_about: string, host_response_time: string, host_response_rate: string, host_acceptance_rate: string, host_is_superhost: string, host_thumbnail_url: string, host_picture_url: string, host_neighbourhood: string, host_listings_count: double, host_total_listings_count: double, host_verifications: string, host_has_profile_pic: string, host_identity_verified: string, street: string, neighbourhood: string, neighbourhood_cleansed: string, neighbourhood_group_cleansed: string, city: string, state: string, zipcode: string, market: string, smart_location: string, country_code

In [9]:
# create correlation between all column and the target column price and show result in dataframe 

# first get list of all column names 
col_names = [columnName.name for columnName in airBnBData_v1.schema.fields if columnName.dataType == DoubleType()]
display(col_names)
# second see if the target column has zero or null values 
airBnBData_v1 = airBnBData_v1.filter((col('bedrooms') > 0 ) & (col('minimum_nights') <= 365) & (col('price').isNotNull()))
print(airBnBData_v1.count())

['id',
 'host_id',
 'host_listings_count',
 'host_total_listings_count',
 'latitude',
 'longitude',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'square_feet',
 'guests_included',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'calculated_host_listings_count',
 'reviews_per_month']

3438


In [11]:
import folium
from folium.plugins import HeatMap

# Create a base map centered on San Francisco
sf_coordinates = (37.7749, -122.4194)
m = folium.Map(location=sf_coordinates, zoom_start=13)

# Sample data for heatmap

lat_long_price_values = airBnBData_v1.select(col("latitude"), col("longitude"), col("price")/600).collect()

lat_long_price_strings = [f"[{lat}, {long}, {price}]" for lat, long, price in lat_long_price_values]

v = ",\n".join(lat_long_price_strings)

heatmap_data = [
    [37.7749, -122.4194],  # San Francisco city center
    [37.7805, -122.4324],
    [37.7858, -122.4395],
    # ... Add more coordinates here
]

# Create a heatmap layer
heatmap_layer = HeatMap(v)

# Add the heatmap layer to the map
heatmap_layer.add_to(m)

# Display the map
m


23/08/24 09:18:05 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

ValueError: Expected two (lat, lon) values for location, instead got: '['.

In [13]:
# after doing some cleaning we can implemnt correlation to see which the most valuable column on our regression 
from pyspark.sql import Row
# col_names['bedrooms']
airBnBData_v1 = airBnBData_v1.withColumn("price", translate(col("price"), "$,", "").cast("double"))
correlation_list = []
df = airBnBData_v1.na.drop(subset=['bedrooms', 'price'])
correlation = df.stat.corr('bedrooms', 'price')
print(correlation)
for col_name in col_names:
    correlation = airBnBData_v1.stat.corr(col_name, "price")
    correlation_list.append(Row(Column=col_name, Correlation=correlation))

# Create a new DataFrame from the list
correlation_dataframe = spark.createDataFrame(correlation_list)
# get the column name that has high correlation to use it with 



0.6770108030249608


In [14]:
# Filter products where the correlation  is more than 50%
filtered_price_correlation = correlation_dataframe.filter(col("Correlation") > 0.50)

# Collect the filtered products into a Python list
filtered_price_list = []
filtered_price_list = [product["Column"] for product in filtered_price_correlation.collect()]


In [15]:



lat_long_price_values = airBnBData_v1.select(col("latitude"), col("longitude"), col("price")/600).collect()

lat_long_price_strings = [f"[{lat}, {long}, {price}]" for lat, long, price in lat_long_price_values]

v = ",\n".join(lat_long_price_strings)

# DO NOT worry about what this HTML code is doing! We took it from Stack Overflow :-)
display(HTML("""
<html>
<head>
 <link rel="stylesheet" href="https://unpkg.com/leaflet@1.3.1/dist/leaflet.css"
   integrity="sha512-Rksm5RenBEKSKFjgI3a41vrjkw4EVPlJ3+OiI65vTjIdo9brlAacEuKOiQ5OFh7cOI1bkDwLqdLw3Zg0cRJAAQ=="
   crossorigin=""/>
 <script src="https://unpkg.com/leaflet@1.3.1/dist/leaflet.js"
   integrity="sha512-/Nsx9X4HebavoBvEBuyp3I7od5tA0UzAxs+j83KgC8PU0kgB4XiK4Lfe4y4cgBtaRJQEIFCW+oC506aPT2L1zw=="
   crossorigin=""></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.heat/0.2.0/leaflet-heat.js"></script>
</head>
<body>
    <div id="mapid" style="width:700px; height:500px"></div>
  <script>
  var mymap = L.map('mapid').setView([37.7587,-122.4486], 12);
  var tiles = L.tileLayer('http://{s}.tile.osm.org/{z}/{x}/{y}.png', {
    attribution: '&copy; <a href="http://osm.org/copyright">OpenStreetMap</a> contributors',
}).addTo(mymap);
  var heat = L.heatLayer([""" + v + """], {radius: 25}).addTo(mymap);
  </script>
  </body>
  </html>
"""))

In [16]:
from pyspark.sql.functions import count
roomsByNeighbourhood = airBnBData_v1.groupBy('neighbourhood').agg(count('bedrooms').alias('bedroomsCount')).orderBy(col('bedroomsCount').desc())
display(roomsByNeighbourhood.show())

+--------------------+-------------+
|       neighbourhood|bedroomsCount|
+--------------------+-------------+
|                null|          368|
|        Capitol Hill|          284|
|             Ballard|          199|
|               Minor|          180|
|          Queen Anne|          176|
|            Belltown|          161|
|             Fremont|          139|
|         Wallingford|          132|
|   North Beacon Hill|           92|
|             Stevens|           91|
| University District|           87|
|          First Hill|           87|
|           Greenwood|           77|
|Central Business ...|           71|
|    Lower Queen Anne|           70|
|             Ravenna|           57|
|       Columbia City|           55|
|            Magnolia|           54|
|       Phinney Ridge|           53|
|          Green Lake|           49|
+--------------------+-------------+
only showing top 20 rows



None

In [17]:
display(airBnBData_v1.select("price").describe().show())

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              3438|
|   mean|130.64485165794068|
| stddev| 93.98820855335254|
|    min|              20.0|
|    max|            1000.0|
+-------+------------------+



None

In [18]:
# add the price to the list 
new_list = ['property_type', 'room_type', 'bed_type', 'neighbourhood','price']
pickedList = filtered_price_list + new_list

regressionDataFrame = airBnBData_v1.select(pickedList)

In [19]:
regressionDataFrame.orderBy(col('price').desc()).show()

+------------+---------+--------+----+-------------+---------------+--------+-----------------+------+
|accommodates|bathrooms|bedrooms|beds|property_type|      room_type|bed_type|    neighbourhood| price|
+------------+---------+--------+----+-------------+---------------+--------+-----------------+------+
|         4.0|      1.0|     2.0| 2.0|        House|Entire home/apt|Real Bed|          Ravenna|1000.0|
|         4.0|      1.0|     1.0| 1.0|    Apartment|Entire home/apt|Real Bed|         Belltown| 999.0|
|        11.0|      4.5|     5.0| 7.0|        House|Entire home/apt|Real Bed|       Queen Anne| 975.0|
|         6.0|      3.5|     3.0| 3.0|        House|Entire home/apt|Real Bed|         Magnolia| 950.0|
|        10.0|      2.0|     3.0| 4.0|    Apartment|Entire home/apt|Real Bed|         Belltown| 899.0|
|         8.0|      4.0|     4.0| 6.0|         Boat|Entire home/apt|Real Bed|         Westlake| 775.0|
|        10.0|      4.0|     6.0| 7.0|        House|Entire home/apt|Real 

In [23]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy="median", inputCols=filtered_price_list, outputCols=filtered_price_list)

imputer_model = imputer.fit(regressionDataFrame)
imputed_df = imputer_model.transform(regressionDataFrame)
imputed_df.show()

+------------+---------+--------+----+-------------+---------------+--------+-------------+-----+
|accommodates|bathrooms|bedrooms|beds|property_type|      room_type|bed_type|neighbourhood|price|
+------------+---------+--------+----+-------------+---------------+--------+-------------+-----+
|         4.0|      1.0|     1.0| 1.0|    Apartment|Entire home/apt|Real Bed|   Queen Anne| 85.0|
|         4.0|      1.0|     1.0| 1.0|    Apartment|Entire home/apt|Real Bed|   Queen Anne|150.0|
|        11.0|      4.5|     5.0| 7.0|        House|Entire home/apt|Real Bed|   Queen Anne|975.0|
|         6.0|      2.0|     3.0| 3.0|        House|Entire home/apt|Real Bed|   Queen Anne|450.0|
|         2.0|      1.0|     1.0| 1.0|        House|   Private room|Real Bed|   Queen Anne|120.0|
|         2.0|      1.0|     1.0| 1.0|        House|   Private room|Real Bed|   Queen Anne| 80.0|
|         2.0|      1.0|     1.0| 1.0|        Cabin|   Private room|Real Bed|   Queen Anne| 60.0|
|         2.0|      

In [25]:
from delta import *

# path = '/Users/badr/Desktop/python/MLpySpark'
# imputed_df.write.format("delta").mode("overwrite").save('/Users/badr/Desktop/python/MLpySpark')

#split data into training and testing data 
train_df, test_df = imputed_df.randomSplit([.8, .2], seed=42)

In [45]:
from pyspark.ml import Pipeline
import mlflow
import mlflow.spark
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
with mlflow.start_run(run_name='LR-single-feat-new') as run :
    vec_ass = VectorAssembler(inputCols=['accommodates'], outputCol='features')
    lr = LinearRegression(featuresCol='features', labelCol='price')
    pipeline=Pipeline(stages=[vec_ass,lr])
    pipeline_model = pipeline.fit(train_df)
    
    # Log parameters
    mlflow.log_param("label", "price")
    mlflow.log_param("features", "accommodates")

    # Log model
    mlflow.spark.log_model(pipeline_model, "newaccommodates", input_example=train_df.limit(5).toPandas()) 

    # Evaluate predictions
    pred_df = pipeline_model.transform(test_df)
    regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="r2")
    regression_evaluator1 = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
    rmse = regression_evaluator1.evaluate(pred_df)
    r2 = regression_evaluator.evaluate(pred_df)

    # Log metrics
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("rmse", rmse)

23/08/23 15:38:46 WARN Instrumentation: [5905300d] regParam is zero, which might cause numerical instability and overfitting.


In [63]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.search_experiments()

experiment_id = run.info.experiment_id
print(experiment_id)
runs_df = mlflow.search_runs(experiment_id)
print(runs_df)
print(runs_df.artifact_uri[0])

artifact_uri = run.info.artifact_uri
model_path = artifact_uri + "/newaccommodates"
loaded_model = mlflow.spark.load_model(model_path)


0
Empty DataFrame
Columns: [run_id, experiment_id, status, artifact_uri, start_time, end_time]
Index: []


KeyError: 1

In [71]:
lr_run = client.get_run('26d6c8d85db04476ae89c921d94115bc')  # Replace 'lr_run_id' with the actual run ID
dt_run = client.get_run('5779db8370264cd4a1be49708a1dfd58')  # 

lr_r2 = lr_run.data.metrics["rmse"]
dt_r2 = dt_run.data.metrics["rmse"]

print(lr_r2 , dt_r2)

92.0368790898503 91.10111707347875


In [ ]:
best_model = None
all_runs = client.search_runs(experiment_ids=['0'])
best_r2 = -float("inf")  # Initialize with negative infinity

# Iterate through all runs
for run_info in all_runs:
    run_id = run_info.run_id
    run = client.get_run(run_id)
    
    # Check if 'r2' metric is available for this run
    if 'r2' in run.data.metrics:
        r2 = run.data.metrics['r2']
        
        # Compare R-squared values
        if r2 > best_r2:
            best_r2 = r2
            best_model = run.info.artifact_uri + "/model"
            
# Load the best model
if best_model:
    loaded_model = mlflow.spark.load_model(best_model)
    print("Best Model:", loaded_model)
else:
    print("No model with R-squared metric found.")

In [73]:
all_runs = client.search_runs(experiment_ids=['0'])
print(all_runs)

# Iterate through all runs and delete artifacts and run information
for run in all_runs:
    run_id = run.info.run_id
    client.delete_run(run_id)

[]


NameError: name 'airBnBData_v1' is not defined